In [0]:
import pandas as pd
import os
import xgboost as xgb
import numpy as np
import gc
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from xgboost import XGBClassifier

from google.colab import drive

In [2]:
drive.mount('/content/gdrive')
label_encoder = LabelEncoder()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Function to find column that have more than 85% missing valuse and delete them
def missing_values(data):
        # Total missing values
        mis_val = data.isnull().sum()
        # Percentage of missing values
        mis_val_percent = 100 * data.isnull().sum() / len(data)
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table=mis_val_table.sort_values(by=[1],ascending=False)
        miss_column_names=[]
        for i in range(len(mis_val_table)+1):
          if mis_val_table.iloc[i,1] >85:
             miss_column_names.append(mis_val_table.index[i])
          else:
            break
        data.drop(mis_val_table.index[i], inplace=True, axis=1)

        return data

In [0]:
# Function to encoding Data Frames
def lable_encoding(data):
    for i in data:
        if data[i].dtype == 'object':
           data[i],_= pd.factorize(data[i])
    return data


In [0]:
#Function to create statistical features
def agg_functions(data):
 stat_function=['count','sum','max','min','std','mean']
 data_stats = data.groupby(data.columns[0]).agg(stat_function)
 data_stats.columns = ["_".join(x) for x in data_stats.columns.ravel()]
 data_stats=data_stats.iloc[:,0:len(data_stats.columns)].reset_index()
 return data_stats


In [0]:
# Read CSV files and creat their dataframes for finding new manual and statistical features ,and also encoding them .

In [0]:
#Read application_test and train and join them
application_test = pd.read_csv('/content/gdrive/My Drive/data/application_test.csv')
application_train = pd.read_csv('/content/gdrive/My Drive/data/application_train.csv')
target = application_train['TARGET']
del(application_train['TARGET'])

app_train_test = application_train.append(application_test, ignore_index = True)
del(application_train,application_test)


In [0]:
#cleanind data of DAYS_EMPLOYED and DAYS_LAST_PHONE_CHANGE columns

app_train_test['DAYS_EMPLOYED'].fillna(365243, inplace = True)
app_train_test['DAYS_LAST_PHONE_CHANGE'].fillna(0, inplace = True)

#Manual Feature Engineering

#RATIO_OF_EMPLOYED_BIRTH: The ratio of client's experience and client' age
app_train_test['RATIO_OF_EMPLOYED_BIRTH']=app_train_test['DAYS_EMPLOYED']/app_train_test['DAYS_BIRTH']

#RATIO_OF_TOTALINCOME_EMPLOYED: The ratio of client's income and client's total income
app_train_test['RATIO_OF_TOTALINCOME_EMPLOYED'] = app_train_test['AMT_INCOME_TOTAL'] / app_train_test['DAYS_EMPLOYED']

#RATIO_OF_CREDIT_TOTALINCOME: The ratio of loan and total income
app_train_test['RATIO_OF_CREDIT_TOTALINCOME'] = app_train_test['AMT_CREDIT'] / app_train_test['AMT_INCOME_TOTAL']

#RATIO_OF_MEMBERS_TOTALINCOME: The ratio of the number of family members and total income
app_train_test['RATIO_OF_MEMBERS_TOTALINCOME']=app_train_test['AMT_INCOME_TOTAL'] / (1 + app_train_test['CNT_FAM_MEMBERS'])

#RATIO_OF_TOTALINCOME_BIRTH: The ratio of total income and client's age
app_train_test['RATIO_OF_TOTALINCOME_BIRTH'] = app_train_test['AMT_INCOME_TOTAL'] / app_train_test['DAYS_BIRTH']

#RATIO_OF_CREDIT_GOODS: The ratio of amount of credit and goods price
app_train_test['RATIO_OF_CREDIT_GOODS']=app_train_test['AMT_CREDIT'] / app_train_test['AMT_GOODS_PRICE']

#RATIO_OF_ANNUITY_TOTALINCOME': the ratio of annuity and total income
app_train_test['RATIO_OF_ANNUITY_TOTALINCOME']=app_train_test['AMT_ANNUITY'] / app_train_test['AMT_INCOME_TOTAL']
app_train_test['RATIO_OF_CREDIT_ANNUITY'] = app_train_test['AMT_CREDIT'] / app_train_test['AMT_ANNUITY']

#RATIO_OF_CAR_BIRTH:The rate of OWN_CAR_AGE /DAYS_BIRTH
app_train_test['RATIO_OF_CAR_EMPLOYED'] = app_train_test['OWN_CAR_AGE'] / app_train_test['DAYS_EMPLOYED']
app_train_test['RATIO_OF_CAR_BIRTH'] = app_train_test['OWN_CAR_AGE'] / app_train_test['DAYS_BIRTH']

In [0]:
#calculate statistical features with agg_function on FLAG_DOCUMENT_2 to FLAG_DOCUMENT_22
flage_docs=[]
for i in range(2,22):
    flage_docs.append('FLAG_DOCUMENT_'+str(i))

app_train_test['flage_docs_KURT'] = app_train_test[flage_docs].kurtosis(axis=1)
app_train_test['flage_docs_SUM']=app_train_test[flage_docs].sum(axis=1)
app_train_test['flage_docs_MEAN']=app_train_test[flage_docs].mean(axis=1)


#EXT_SOURCE_1 & 2 & 3 :Normalized score from external data source
stat_function=[]
stat_function=['count','sum','max','min','var','std','mean']
EXT_SOURCE_123=[]
EXT_SOURCE_123=['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']
app_train_test['EXT_SOURCE_123_MUl']=app_train_test['EXT_SOURCE_1'] * app_train_test['EXT_SOURCE_2'] * app_train_test['EXT_SOURCE_3']
for a in stat_function:
    app_train_test['EXT_SOURCE_123'+'_'+str(a).upper()]=app_train_test[EXT_SOURCE_123].agg(a,axis=1)

# encoding test and train sets
app_train_test=lable_encoding(app_train_test)
# Drop some unnecessarily features
columns = ['COMMONAREA_MODE','ELEVATORS_MEDI', 'EMERGENCYSTATE_MODE','FLAG_EMP_PHONE', 'FLAG_MOBIL', 
           'FLAG_CONT_MOBILE','FLOORSMIN_MODE','FONDKAPREMONT_MODE','FLOORSMAX_MODE','HOUSETYPE_MODE',
           'FLAG_EMAIL', 'FLAG_PHONE','LIVE_REGION_NOT_WORK_REGION', 'LIVINGAPARTMENTS_MODE','LIVINGAREA_MODE',
           'NAME_TYPE_SUITE', 'NONLIVINGAPARTMENTS_MODE','NONLIVINGAREA_MODE', 'WALLSMATERIAL_MODE']

app_train_test.drop(columns, inplace=True, axis=1)
app_train_test.drop(flage_docs, inplace=True, axis=1)

    

In [0]:
#Read bureau_balance , encoding it and calculate statistical features with agg_function
bureau_balance = pd.read_csv('/content/gdrive/My Drive/data/bureau_balance.csv')
bureau_balance_stats=agg_functions(lable_encoding(bureau_balance))

#Read bureau_balance ,calculate manual features, encoding it and calculate statistical features with agg_function
bureau = pd.read_csv('/content/gdrive/My Drive/data/bureau.csv')

# CREDIT_ENDDATE:differences between remaining duration of CB credit  at the time of application in Home credit and days since CB credit ended at the time of application in Home Credit 
bureau['CREDIT_ENDDATE'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']

# RATIO_OF_CREDI:The ratio of current credit amount for the credit bureau credit / current debt on Credit Bureau credit
bureau['RATIO_OF_CREDIT'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_CREDIT_SUM_DEBT']

# RATIO_OF_AMT_ANNUITY:The ratio of Current credit amount for the Credit Bureau credit / Annuity of the Credit Bureau credit
bureau['RATIO_OF_AMT_ANNUITY'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_ANNUITY']

# join bureau_balance and bureau
bureau_balance_stats = pd.merge(left = bureau, right = bureau_balance_stats, left_on='SK_ID_BUREAU', right_on='SK_ID_BUREAU', how = 'left')
bureau_balance_stats.drop(columns = ['SK_ID_BUREAU'], inplace = True)

bureau_balance_stats=agg_functions(lable_encoding(bureau_balance_stats))

# join bureau_balance_stats and app_train_test
app_train_test = pd.merge(left=app_train_test, right=bureau_balance_stats, on='SK_ID_CURR', how='left')

del bureau_balance_stats,bureau


In [0]:
#Read credit_card_balance , encoding it and calculate statistical features with agg_function
credit_card_balance=pd.read_csv('/content/gdrive/My Drive/data/credit_card_balance.csv')
credit_card_balance.drop(columns = ['SK_ID_PREV'], inplace = True)

# RATI_OF_PAYMENT:The ratio of amount of the client pay during the month on the previous credit / minimal installment for this month of the previous credit
credit_card_balance['RATI_OF_PAYMENT'] = credit_card_balance['AMT_PAYMENT_CURRENT'] / credit_card_balance['AMT_INST_MIN_REGULARITY']

# RATIO_OF_ACTUAL_DRAWINGS:The ratio of amount drawing at ATM during the month of the previous credit / credit card limit during the month of the previous credit
credit_card_balance['RATIO_OF_ACTUAL_DRAWINGS'] = credit_card_balance['AMT_DRAWINGS_ATM_CURRENT'] / credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL']

credit_card_balance_stats=agg_functions(lable_encoding(credit_card_balance))

# join credit_card_balance_stats and app_train_test
app_train_test = pd.merge(left=app_train_test, right=credit_card_balance_stats, on='SK_ID_CURR', how='left')

del(credit_card_balance,credit_card_balance_stats)


In [0]:
#Read installments_payments , encoding it and calculate statistical features with agg_function

installments_payments = pd.read_csv('/content/gdrive/My Drive/data/installments_payments.csv')
installments_payments.drop(columns = ['SK_ID_PREV'], inplace = True)

# PAYMENT_INSTALMENT: the rate of previous credit installment /  amount of previous credit installment
installments_payments['PAYMENT_INSTALMENT'] = installments_payments['AMT_PAYMENT'] / installments_payments['AMT_INSTALMENT'] 

# AMT_DIVERSITY: difference between amount of previous credit installment and the rate of previous credit installment
installments_payments['AMT_DIVERSITY'] = installments_payments['AMT_INSTALMENT'] - installments_payments['AMT_PAYMENT'] 

installments_payments_stats=agg_functions(lable_encoding(installments_payments))

# join installments_payments_stats and app_train_test
app_train_test = pd.merge(left=app_train_test, right=installments_payments_stats, on='SK_ID_CURR', how='left')

del(installments_payments,installments_payments_stats)


In [0]:
#Read POS_CASH_balance , encoding it and calculate statistical features with agg_function

POS_CASH_balance = pd.read_csv('/content/gdrive/My Drive/data/POS_CASH_balance.csv')
POS_CASH_balance.drop(columns = ['SK_ID_PREV'], inplace = True)
POS_CASH_balance_stats=agg_functions(lable_encoding(POS_CASH_balance))

# join POS_CASH_balance_stats and app_train_test
app_train_test = pd.merge(left=app_train_test, right=POS_CASH_balance_stats, on='SK_ID_CURR', how='left')

del(POS_CASH_balance,POS_CASH_balance_stats)


In [0]:
#Read previous_application , encoding it and calculate statistical features with agg_function

previous_application = pd.read_csv('/content/gdrive/My Drive/data/previous_application.csv')
previous_application.drop(columns = ['SK_ID_PREV'], inplace = True)

# Manual Feature Generation
#DIVERSITY_APP_CREDIT: Difference between  the number of client ask on the previous application and final credit amount on the previous application
previous_application['DIVERSITY_APP_CREDIT'] = previous_application['AMT_APPLICATION'] - previous_application['AMT_CREDIT']

#RATIO_OF_APP_CREDIT:The rate of number of client ask on the previous application / final credit amount on the previous application
previous_application['RATIO_OF_APP_CREDIT'] = previous_application['AMT_APPLICATION'] / previous_application['AMT_CREDIT']

#RATIO_OF_CRE_ANNULIY:The rate of final credit amount on the previous application / Annuity of previous application
previous_application['RATIO_OF_CRE_ANNULIY'] = previous_application['AMT_CREDIT']/previous_application['AMT_ANNUITY']

#DIVRSITY_OF_DUE:Difference between Relative to application date of current application when was the first due of the previous application
# and Relative to application date of current application when was the last due date of the previous application
previous_application['DIVRSITY_OF_DUE'] = previous_application['DAYS_LAST_DUE_1ST_VERSION'] - previous_application['DAYS_LAST_DUE']

previous_application_stats=agg_functions(lable_encoding(previous_application))

# join previous_application_stats and app_train_test
app_train_test = pd.merge(left=app_train_test, right=previous_application_stats, on='SK_ID_CURR', how='left')

del(previous_application,previous_application_stats)


In [0]:
app_train_test=missing_values(app_train_test)
columns_name = []
for i in range(len(app_train_test.columns)):
    columns_name.append(app_train_test.columns[i] + '_' + str(i))
app_train_test.columns = columns_name


In [0]:
# seperate the independent and target variable on training data

train_data = app_train_test.iloc[:307511, :]
test_data = app_train_test.iloc[307511:, :]

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(train_data, target, test_size=0.15)

In [0]:
# XGBoost Model
param = { 'objective' : "binary:logistic",'booster' : "gbtree",'eval_metric' : 'auc','nthread' : 4,'eta' : 0.03,
          'max_depth' : 5,'min_child_weight' : 40,'gamma' : 10,'subsample' : .8,'colsample_bytree' : 0.5,
          'colsample_bylevel' : 0.5,'alpha' : 0.3,'lambda' : 0,'nrounds' : 1000}

xgtrain = xgb.DMatrix(X_train.values, label=y_train)
xgtest = xgb.DMatrix(X_test.values,label=y_test)
watchlist=[(xgtrain, 'train'),(xgtest, 'eval')]

# fit model
model = xgb.train(param, xgtrain, param['nrounds'], watchlist, verbose_eval = True)

[0]	train-auc:0.711547	eval-auc:0.715611
[1]	train-auc:0.727444	eval-auc:0.726785
[2]	train-auc:0.732041	eval-auc:0.731133
[3]	train-auc:0.734466	eval-auc:0.733283
[4]	train-auc:0.734935	eval-auc:0.734031
[5]	train-auc:0.737357	eval-auc:0.73662
[6]	train-auc:0.739485	eval-auc:0.738639
[7]	train-auc:0.74119	eval-auc:0.740422
[8]	train-auc:0.742084	eval-auc:0.740851
[9]	train-auc:0.742202	eval-auc:0.741034
[10]	train-auc:0.742986	eval-auc:0.741287
[11]	train-auc:0.743911	eval-auc:0.742032
[12]	train-auc:0.744551	eval-auc:0.742564
[13]	train-auc:0.74484	eval-auc:0.742821
[14]	train-auc:0.745485	eval-auc:0.743486
[15]	train-auc:0.745638	eval-auc:0.743734
[16]	train-auc:0.746209	eval-auc:0.744563
[17]	train-auc:0.746507	eval-auc:0.744809
[18]	train-auc:0.746537	eval-auc:0.744947
[19]	train-auc:0.746855	eval-auc:0.745242
[20]	train-auc:0.747134	eval-auc:0.745432
[21]	train-auc:0.747387	eval-auc:0.745648
[22]	train-auc:0.747639	eval-auc:0.745748
[23]	train-auc:0.748534	eval-auc:0.746549
[24]	

In [0]:
# predict testset
column_test_name = []
for i in range(len(test_data.columns)):
    column_test_name.append('f' + str(i))
test_data.columns = column_test_name
xgb_test = xgb.DMatrix(test_data)
xgb_pred  =model.predict(xgb_test)
xgb_pred 

array([0.04108071, 0.09767029, 0.03022283, ..., 0.00823099, 0.03047396,
       0.22360802], dtype=float32)

In [0]:
testset=app_train_test.iloc[307511:, :]
pd_result=pd.DataFrame(columns=['SK_ID_CURR','TARGET'])
pd_result['SK_ID_CURR']=testset['SK_ID_CURR_0']
pd_result['TARGET']=xgb_pred
pd_result=pd_result.set_index('SK_ID_CURR')

##save result to csv file
pd.DataFrame(pd_result).to_csv("/content/gdrive/My Drive/data/home_result.csv")

